In [ ]:
import tensorflow as tf
import os
import sys
import numpy as np
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow
from skimage.transform import resize
import random
import matplotlib.pyplot as plt
import time
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras_unet_collection import models, losses
from google.colab import drive
from skimage import color
import pydicom as dicom

In [ ]:
!pip install keras_unet_collection
!pip install pydicom

In [ ]:
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/My Drive/data-v2/iaaa-data-v3.zip' -d '/content/drive/My Drive/data-v2/data-v2-unzip'

In [ ]:
IMG_WIDTH = 512
IMG_HEIGHT = 256
IMG_CHANNELS = 1

DATA_PATH = '/content/drive/My Drive/data-v2/data-v2-unzip/iaaa-data-v3/DICOM/' # images
Mask_path1= '/content/drive/My Drive/data-v2/data-v2-unzip/iaaa-data-v3/masks/'   # masks


seed = 42
random.seed = seed
np.random.seed = seed
 
image_ids = next(os.walk(DATA_PATH))[2]
mask_ids= next(os.walk(Mask_path1))[2]
# img = imread(DATA_PATH+ image_ids[1])[:,:]
# imshow(img)
# plt.show()
X = np.zeros((len(image_ids), IMG_HEIGHT, IMG_WIDTH))

Y = np.zeros((len(image_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool_)

padding_ = np.zeros((IMG_HEIGHT,IMG_WIDTH))

In [ ]:
print(type(image_ids))
x=image_ids.index('2.25.305861492438344748144578306421271906759.dcm')
# x = np.where(image_ids == '2.25.228745877808304447814199720807861699408.dcm')
print(x)

# preprocess

In [ ]:
for n, id_ in tqdm(enumerate(image_ids), total=len(image_ids)):
  try:
    path = DATA_PATH 
    img = dicom.dcmread(DATA_PATH+ id_)
    img = img.pixel_array[:,:]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    image = img.copy()

    image[:25,:] = padding_[:25,:]
    image[img.shape[0]-25:img.shape[0],:] = padding_[img.shape[0]-25:img.shape[0],:]

    image[:,:50] = padding_[:,:50]
    image[:,img.shape[1]-80:img.shape[1]] = padding_[:,img.shape[1]-80:img.shape[1]]
    X[n] = image
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool_)
    
    for mask_file in next(os.walk(Mask_path1))[2]:
      # print(mask_file[:len(mask_file) - 3])
      # print(id_[:len(id_) - 3])
      if mask_file[:len(mask_file) - 3]==id_[:len(id_) - 3]:
        mask_ = imread(Mask_path1 + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
        mask[:50,:50] = 0    
    Y[n] = mask
  except:
    print('error')
    
x_train=X 
y_train=Y

print('Done!')

image_x = random.randint(0, len(image_ids)-1)
imshow(x_train[image_x],cmap='gray')
plt.show()
imshow(np.squeeze(y_train[image_x]),cmap='gray')
plt.show()

# model

In [ ]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

# Encoding
c1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Decoding
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', losses.dice_coef])
model.summary()

# Train and save model

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('/content/model_3rdm.h5', verbose=1, save_best_only=True)
callbacks= [
            tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
            tf.keras.callbacks.TensorBoard(log_dir='./logs')]
results = model.fit(x_train,y_train, validation_split=0.1, batch_size=16, epochs=30, callbacks=callbacks)
t = time.time()
my_keras_model_filepath = '/content/drive/My Drive/my models/saved_model_577_diffszie_512x256.h5'.format(int(t))
print(my_keras_model_filepath)
model.save(my_keras_model_filepath)

In [ ]:
model_v1 = keras.models.load_model('/content/drive/My Drive/my models/saved_model_577_diffszie_512x256.h5', compile=False)
model_v1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', losses.dice_coef])

In [ ]:
training_loss=results.history['loss']
training_accuracy=results.history['accuracy']
validation_loss=results.history['val_loss']
validation_accuracy=results.history['val_accuracy']

epochs_range=range(len(training_accuracy))
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, training_accuracy, label='Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, training_loss, label='Training Loss')
plt.plot(epochs_range, validation_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Test

In [ ]:
TEST_PATH = '/content/drive/My Drive/Colab Notebooks/test/'

test_ids = next(os.walk(TEST_PATH))[2]

len(test_ids)
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH))

sizes_test=[]
print('resizing training images')
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
  path = TEST_PATH 
  img = dicom.dcmread(path+ id_)
  img = img.pixel_array[:,:]
  sizes_test.append([img.shape[0],img.shape[1]])
  img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  X_test[n]=img 

In [ ]:
preds_test = model_v1.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
ix = random.randint(0, len(preds_test_t))
imshow(X_test[1],cmap='gray')
plt.show()
imshow(np.squeeze(preds_test_t[1]),cmap='gray')
plt.show()

In [ ]:
for i in range(50):
  imshow(np.squeeze(y_train[i]),cmap='gray')
  plt.show()

In [ ]:
# model_v1
image_x = random.randint(46, len(image_ids)-1)
imshow(x_train[image_x],cmap='gray')
plt.show()
imshow(np.squeeze(y_train[image_x]),cmap='gray')
plt.show()